Get images and save them independently as 2D slices

Dependencies

In [6]:
import torch
import torchvision
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torch.utils.data import Dataset
from torchvision.transforms import ToTensor

import matplotlib.pyplot as plt
from matplotlib.widgets import Slider, Button, RadioButtons

import scipy

import SimpleITK as sitk
reader = sitk.ImageFileReader()
reader.SetImageIO("MetaImageIO")

import numpy as np

import os

import pathlib

from natsort import natsorted

#Set GPU/Cuda Device to run model on
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)
print(f"Using {device} device")

np.random.seed(46)

Using cuda device


Dataset Directories

In [7]:
#Toy Dataset Directory from Spider | Grand Challenge 
dummy_train_img_dir = pathlib.Path(r"spider_toy_dset/dummy_train_images")
dummy_train_label_dir = pathlib.Path(r"spider_toy_dset/dummy_train_labels")
dummy_test_img_dir = pathlib.Path(r"spider_toy_dset/dummy_test_images")
dummy_test_label_dir= pathlib.Path(r"spider_toy_dset/dummy_test_labels")

Image Class, Returns 3D Array

In [8]:
from transforms import mri_transforms


#TODO add bool image label for interp 
class Mri:
    def __init__(self, path):
        mri_mha = sitk.ReadImage(path, imageIO = "MetaImageIO") #explicitly setting ioreader just in case
        
       #TODO use sitk permute to change dimensions to 2 0 1 to help with going through slices for interpolating in 2d
        if(mri_mha.GetSize()[0] > mri_mha.GetSize()[1] or mri_mha.GetSize()[0] > mri_mha.GetSize()[2]): #if z axis isn't first
          mri_mha = sitk.PermuteAxes(mri_mha, (2, 0, 1))
          
        #get 3d array from mri
        mri_a = np.array(sitk.GetArrayFromImage(mri_mha)) #mri_array

        #transpose array to format z x y or z y x 
        if(mri_a.shape[0] > mri_a.shape[1] or mri_a.shape[0] > mri_a.shape[2]): #if z axis isn't first
          mri_a = np.transpose(mri_a, (2, 0, 1))

        '''
        if(mri_a.shape[2] > mri_a.shape[1]): #bring images to vertical orientation
          mri_a = np.transpose(mri_a, (0, 2, 1))
        '''
      
        mri_a_float32 = mri_a.astype(dtype = np.float32)
        #TODO: set bounds to [-1000, 2000] https://en.wikipedia.org/wiki/Hounsfield_scale
        self.hu_a = mri_a_float32

Sort Directories

In [9]:
#get lists from directories

#toy dset 

label_dir_list = os.listdir(dummy_train_label_dir)
image_dir_list = os.listdir(dummy_train_img_dir) 

#local dset
'''
image_dir_list = os.listdir(local_img_idr)
label_dir_list = os.listdir(local_label_dir)
'''
#sort lists
image_dir_list = natsorted(image_dir_list)
label_dir_list = natsorted(label_dir_list)
#empty lists to hold x and y dimensions of images
row_list = []
col_list = []

#dirlen = len(os.listdir(dummy_train_label_dir))
dirlen = len(os.listdir(dummy_train_label_dir))

print(dirlen)

#print(local_label_idr)


44


Testing on single image


In [34]:
img_path = dummy_train_img_dir.joinpath(image_dir_list[0])
test_mri = Mri(img_path)
print(test_mri.hu_a.shape)

test_slice_arr = test_mri.hu_a[25, : ,:]
#print(test_slice_arr.shape)

test_slice_sitk = sitk.GetImageFromArray(test_slice_arr)
#print(test_slice_sitk)

dir_split = image_dir_list[0].split(".") #splits 1_t1.mha to 1_1 and .mha
pre = dir_split[0]
print(pre)

post = "_" + str(0) + "." + dir_split[1]
print(post)

dir_path = pre + post
print(dir_path)

file_path = pathlib.Path(r"spider_toy_dset_slices/dummy_train_img_slices")

full_path = file_path.joinpath(dir_path)

print(full_path)

sitk.WriteImage(test_slice_sitk, full_path)

(50, 578, 448)
1_t1
_0.mha
1_t1_0.mha
spider_toy_dset_slices\dummy_train_img_slices\1_t1_0.mha


Remove slices with 0 label value and save the remaining slices in a new directory

In [19]:
from transforms import array_transforms

for idx in range(0, dirlen):
  #print("dirlen", dirlen)
  
 #toy dset 
  
  img_path = dummy_train_img_dir.joinpath(image_dir_list[idx])
  label_path = dummy_train_label_dir.joinpath(label_dir_list[idx])#first part before joinpath is pathlib.Path, second part is the directory of the file 

  print(image_dir_list[idx])
  print(type(image_dir_list[idx]))

  '''
  img_path = local_img_idr.joinpath(image_dir_list[idx])
  label_path = local_label_dir.joinpath(label_dir_list[idx]) #first part before joinpath is pathlib.Path, second part is the directory of the file 
  '''
  image = Mri(img_path)
  label = Mri(label_path)
  

  #print("before", label.hu_a.shape)
  #remove empty slices
  image_a, label_a = array_transforms.remove_empty_slices(image.hu_a, label.hu_a)

  
  #going to apply cropping to another point once the slices are ready
  '''  
  #zero crop
  image_a, label_a = array_transforms.crop_zero(image_a, label_a)
  '''
  '''
  if(image_a.shape[1] > 600): #if image way too high res
    print("high res image in directory", img_path)
  '''
  #print("label after: ", label_a.shape)
  
  row_list.append(image_a.shape[1]) #add x value to list
  col_list.append(image_a.shape[2]) #add y value to list 

  
#calculate max 
row_dim_max = max(row_list)
col_dim_max = max(col_list)

print("row max:", max(row_list))
print("col max:", max(col_list))





1_t1.mha
<class 'str'>
1_t2.mha
<class 'str'>
2_t1.mha
<class 'str'>
2_t2.mha
<class 'str'>
3_t1.mha
<class 'str'>
3_t2.mha
<class 'str'>
4_t1.mha
<class 'str'>
4_t2.mha
<class 'str'>
5_t1.mha
<class 'str'>
5_t2.mha
<class 'str'>
7_t1.mha
<class 'str'>
7_t2.mha
<class 'str'>
8_t1.mha
<class 'str'>
8_t2.mha
<class 'str'>
9_t1.mha
<class 'str'>
9_t2.mha
<class 'str'>
11_t1.mha
<class 'str'>
11_t2.mha
<class 'str'>
12_t1.mha
<class 'str'>
12_t2.mha
<class 'str'>
13_t1.mha
<class 'str'>
13_t2.mha
<class 'str'>
15_t1.mha
<class 'str'>
15_t2.mha
<class 'str'>
16_t1.mha
<class 'str'>
16_t2.mha
<class 'str'>
17_t1.mha
<class 'str'>
17_t2.mha
<class 'str'>
18_t1.mha
<class 'str'>
18_t2.mha
<class 'str'>
19_t1.mha
<class 'str'>
19_t2.mha
<class 'str'>
22_t1.mha
<class 'str'>
22_t2.mha
<class 'str'>
23_t1.mha
<class 'str'>
23_t2.mha
<class 'str'>
24_t1.mha
<class 'str'>
24_t2.mha
<class 'str'>
25_t1.mha
<class 'str'>
25_t2.mha
<class 'str'>
28_t1.mha
<class 'str'>
28_t2.mha
<class 'str'>
29_t1.mh